In [95]:
import pickle
from util_fdp import *

In [96]:
cvg_case2 = load_pickle('case_federated_hete_cvg.pkl')
mae_case2 = load_pickle('case_federated_hete_mae.pkl')

In [97]:
print(cvg_case2)
print(mae_case2)
# {5000: {0.3: {0.9: {'log': {(0.51, 100): np.float64(1.0)}}}}}
# {5000: {0.3: {0.9: {'log': {(0.51, 100): np.float64(0.0037842187136836085)}}}}}

{5000: {0.3: {0.25: {1: np.float64(0.994), 5: np.float64(0.98), 'log': np.float64(1.0)}, 0.9: {1: np.float64(0.988), 5: np.float64(1.0), 'log': np.float64(1.0)}}, 0.5: {0.25: {1: np.float64(0.992), 5: np.float64(1.0), 'log': np.float64(1.0)}, 0.9: {1: np.float64(0.924), 5: np.float64(0.996), 'log': np.float64(1.0)}}, 0.8: {0.25: {1: np.float64(0.997), 5: np.float64(1.0), 'log': np.float64(1.0)}, 0.9: {1: np.float64(0.952), 5: np.float64(0.993), 'log': np.float64(0.983)}}}, 50000: {0.3: {0.25: {1: np.float64(1.0), 5: np.float64(0.915), 'log': np.float64(0.999)}, 0.9: {1: np.float64(0.87), 5: np.float64(0.976), 'log': np.float64(0.954)}}, 0.5: {0.25: {1: np.float64(0.987), 5: np.float64(0.91), 'log': np.float64(0.488)}, 0.9: {1: np.float64(0.956), 5: np.float64(0.98), 'log': np.float64(0.991)}}, 0.8: {0.25: {1: np.float64(0.997), 5: np.float64(0.915), 'log': np.float64(1.0)}, 0.9: {1: np.float64(0.895), 5: np.float64(0.945), 'log': np.float64(0.181)}}}}
{5000: {0.3: {0.25: {1: np.float64

In [98]:
import pandas as pd

cvg_log = cvg_case2[5000][0.8][0.25]['log']
mae_log = mae_case2[5000][0.8][0.25]['log']
cvg_5   = cvg_case2[5000][0.8][0.25][5]
mae_5   = mae_case2[5000][0.8][0.25][5]

rows = []
for E, cvg_map, mae_map in [('log', cvg_log, mae_log), (5, cvg_5, mae_5)]:
# for E, cvg_map, mae_map in [(5, cvg_5, mae_5)]:
# for E, cvg_map, mae_map in [('log', cvg_log, mae_log)]:
    for (a, b), cvg in cvg_map.items():
        rows.append({
            "E": E,
            "a": float(a),
            "b": float(b),
            "cvg": float(cvg),
            "mae": float(mae_map[(a, b)])
        })

df = pd.DataFrame(rows)
df = df.sort_values(by=["E","a","b"]).reset_index(drop=True)


print(df.to_string(index=False))

# df.to_csv("combined_cvg_mae_T5000_tau0.3_rs0.9.csv", index=False)


AttributeError: 'numpy.float64' object has no attribute 'items'

In [99]:
def generate_combined_table(data_ci, data_mae, caption="Results",
                            label="tab:combined"):
    # 自动获取所有n值并按数字顺序排序
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())), key=int)
    latex_code = f""
    for n in all_n:
        # 添加n标题行
        latex_code += f"\\multicolumn{{6}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"
        
        # 遍历所有tau和r组合
        for tau in sorted(data_ci[n].keys(), key=float):
            for r_val in data_ci[n][tau].keys():
                # 生成行内容
                row = f"{tau} & {r_val if isinstance(r_val, str) else r_val} "
                
                # 遍历所有E值
                for e in data_ci[n][tau][r_val].keys():
                    ci = data_ci[n][tau][r_val].get(e, "--")
                    mae = data_mae[n][tau][r_val].get(e, "--")
                    cell = f"& {ci:.3f}({mae:.4f}) " if isinstance(ci, float) and isinstance(mae, float) else "& -- "
                    row += cell
                
                latex_code += row + "\\\\\n"
            
            # 添加tau之间的分隔线
            latex_code += "\\hline\n"
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table}"""
    return latex_code

In [100]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open("combined_table.tex", "w") as f:
    f.write(latex_output)

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [20]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [3]:

# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train_func(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test_func(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total

In [ ]:
import os
import tempfile

from ray.tune import Checkpoint

def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train_func(model, optimizer, train_loader)
        acc = test_func(model, test_loader)


        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            checkpoint = None
            if (i + 1) % 5 == 0:
                # This saves the model to the trial directory
                torch.save(
                    model.state_dict(),
                    os.path.join(temp_checkpoint_dir, "model.pth")
                )
                checkpoint = Checkpoint.from_directory(temp_checkpoint_dir)

            # Send the current training result back to Tune
            tune.report({"mean_accuracy": acc}, checkpoint=checkpoint)


In [5]:
search_space = {
    "lr": tune.sample_from(lambda spec: 10 ** (-10 * np.random.rand())),
    "momentum": tune.uniform(0.1, 0.9),
}

# Uncomment this to enable distributed execution
# `ray.init(address="auto")`

# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

tuner = tune.Tuner(
    train_mnist,
    param_space=search_space,
)
results = tuner.fit()

(train_mnist pid=234083) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_mnist_2025-04-29_02-27-52/train_mnist_8f5a1_00000_0_momentum=0.6544_2025-04-29_02-27-56/checkpoint_000000)
(train_mnist pid=234083) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_mnist_2025-04-29_02-27-52/train_mnist_8f5a1_00000_0_momentum=0.6544_2025-04-29_02-27-56/checkpoint_000001)
2025-04-29 02:28:02,284	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_mnist_2025-04-29_02-27-52' in 0.0089s.
2025-04-29 02:28:02,307	INFO tune.py:1041 -- Total run time: 6.14 seconds (6.06 seconds for the tuning loop).
